# Splitting Wells Machine

In [ ]:
import pandas as pd
import numpy as np

# initialising marker
marker = pd.read_excel(r"C:\Users\GG\Downloads\MARKER MLQRM 2.xlsx", 
                       sheet_name = "Sheet2")
marker = pd.pivot_table(marker, 
                        index = "Well", 
                        columns = "Surface", 
                        values = "MD")
marker = marker.reset_index().rename_axis(None, axis = 1)

# completion event
perfo = pd.read_excel(r"C:\Users\GG\Downloads\Completion Final 2022 (1).xlsx", 
                      sheet_name = "Raw (Hapus Plug Squeeze)")
perfo.drop(['WELL','CONF','COMP'], axis=1, inplace=True)

# list of marker
marker = marker[['Well', 'TETX', 'TET1', 'TET2', 'TET3', 'TET4', 'TET5', 'TET6', 'TET7',
       'BKX', 'BKA1', 'A1U2', 'A1U1', 'A1L1', 'BKA2', 'A2U1', 'BNB1', 'B1U2',
       'B1U1', 'BNB2', 'B2U3', 'B2U2', 'B2U1', 'BND', 'DU4', 'DU3', 'DU2', 'DU1', 'MNS']]

# list of targeted well (1915 wells)
all_well = pd.read_excel(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx", 
                         sheet_name = 'Well')['Well'].tolist()

# filtering marker and perdo data dor only 13 wells
marker_wells = marker[marker['Well'].isin(all_well)]
marker_wells = marker_wells.set_index('Well')
perfo_wells =  perfo[perfo['UWI'].isin(all_well)]

# merging the marker and perfo data
perfo_wells = perfo_wells.merge(marker_wells, 
                                      how = 'left', 
                                      left_on = 'UWI', 
                                      right_on = 'Well')

# list of sands
sand = ['TETX', 'TET1', 'TET2', 'TET3', 'TET4', 'TET5', 'TET6', 'TET7',
       'BKX', 'BKA1', 'A1U2', 'A1U1', 'A1L1', 'BKA2', 'A2U1', 'BNB1', 'B1U2',
       'B1U1', 'BNB2', 'B2U3', 'B2U2', 'B2U1', 'BND', 'DU4', 'DU3', 'DU2', 'DU1', 'MNS']

# dropping some unuseful columns
perfo_wells = perfo_wells.drop(['SELISIH'], axis = 1)

In [ ]:
# Markering Telisa and Sihapas
import warnings
warnings.filterwarnings("ignore")

# blank dataframe to contain marker from top completion and bottom completion
df = pd.DataFrame()
# start markering!
for x,y in list(zip(['TOP', 'DEPTH'], ['TOP_WAY', 'BOTTOM_WAY'])):
    # initialising iterator
    i = 0
    # looping through 13 wells
    for well in all_well:
        # initialising temporary dataframe
        temp = perfo_wells[perfo_wells['UWI'] == well]
        temp = temp.dropna(axis = 1)
        sand_list = list(temp.columns[5:])
        
        j = 0
        while j < len(temp):

            for k in range(0, len(sand_list)):
                
                # adressing one marker only
                if len(sand_list) == 1:
                    if temp[x][i+j] >= temp[sand_list[k]][i+j]:
                        df.loc[i+j,y] = str(sand[sand.index(sand_list[k])])
                        break
                    else:
                        df.loc[i+j,y] = str(sand[sand.index(sand_list[k]) - 1])
                        break
                else:
                    # addressing first sand
                    if sand_list[k] == sand_list[0]:
                        if temp[x][i+j] < temp[sand_list[k]][i+j]:
                            df.loc[i+j,y] = str(sand[sand.index(sand_list[k]) - 1])
                            break
                
                    # addressing last sand
                    elif sand_list[k] == sand_list[-1]:

                        if temp[x][i+j] >= temp[sand_list[k]][i+j]:
                            df.loc[i+j,y] = str(sand_list[sand_list.index(sand_list[k])])
                            break
                        else:
                            df.loc[i+j,y] = str(sand_list[sand_list.index(sand_list[k]) - 1])
                            break
                    
                    # addressing all sand except first and last sand
                    else:
                        if temp[x][i+j] < temp[sand_list[k]][i+j]:
                            df.loc[i+j,y] = str(sand_list[sand_list.index(sand_list[k]) - 1])
                            break
            
            j += 1 
        i += j

In [ ]:
# dropping marker columns and adding TOP_WAY and BOTTOM_WAY 
perfo_wells['TOP_WAY'] = df['TOP_WAY']
perfo_wells['BOTTOM_WAY'] = df['BOTTOM_WAY']
perfo_wells['MARKER'] = np.nan # containing final marker

i = 0
# markering final marker
for well in all_well:
    
    # initialising temporary dataframe
    temp = perfo_wells[perfo_wells['UWI'] == well]
    temp = temp.dropna(axis = 1)
    temp.drop(['TOP_WAY', 'BOTTOM_WAY'], axis=1, inplace=True)
    sand_list = list(temp.columns[5:])
    
    #combining TOP_WAY and BOTTOM_WAY
    j = 0
    while j < len(temp):
        
        if (
            (perfo_wells['TOP_WAY'][i+j] not in sand_list and  perfo_wells['BOTTOM_WAY'][i+j] not in sand_list) or 
            (perfo_wells['TOP_WAY'][i+j] not in sand_list or perfo_wells['BOTTOM_WAY'][i+j] not in sand_list)
            ):
            perfo_wells['MARKER'][i+j] = sand[sand.index(perfo_wells['TOP_WAY'][i+j]):sand.index(perfo_wells['BOTTOM_WAY'][i+j])+1]
        else:
            perfo_wells['MARKER'][i+j] = sand_list[sand_list.index(perfo_wells['TOP_WAY'][i+j]):sand_list.index(perfo_wells['BOTTOM_WAY'][i+j])+1]

        j += 1
    
    i += j

# sorting the completion data
perfo_wells = perfo_wells.sort_values(by = ['UWI', 'DATE', 'STATUS']).reset_index(drop=True)

In [ ]:
# function for writing perforation thickness for each sand on the production data in the excel format
def writing_perforation (sands, i):

    x = 0
    for key, value in sands.items():
        if value > 0:
            prod.range(i, 16 + x).value = value
        x += 1

# function to count perforated sand at certain event of production 
def count_sand(sand, pf):

    if len(pf) > 0:
        
        x = 0
        for i in range(0, len(pf)):
            if sand in pf[i]:
                x += 1        
        return x

    else:
        return 0

def squeeze_machine(tp,bp,pf,pd,ts,bs,sf,sd):
    
    a = 0
    # looping through every squeezed event   
    while a < len(sf):

        #checking each squeeze event to each perforation event
        b = 0
        while b < len (pf):
        
            # handling partial squeeze
            if (sd[a] >= pd[b]):
                
                # changing top perforation and deleting squeezed sand
                if ((ts[a] == tp[b]) and  bs[a] < bp[b]):
                    tp[b] = bs[a]
                    if (len(pf[b]) >= len(sf[a])):
                        pf[b] = [x for x in pf[b] if x not in sf[a]]

                # changing bottom perforation and deleting squeezed sand
                elif ((ts[a] > tp[b]) and  bs[a] == bp[b]):
                    bp[b] = ts[a]
                    if (len(pf[b]) >= len(sf[a])):
                        pf[b] = [x for x in pf[b] if x not in sf[a]]

            # deleting perforation event (date, top perforation, bottom perforation and sand) that squeezed
            if ((ts[a] == tp[b] and bs[a] == bp[b]) 
                and sd[a] >= pd[b]):
                del pf[b]
                del tp[b]
                del bp[b]
                del pd[b] 
            else:
                b += 1

        a += 1
    
    # empting the unuseful list
    pd, ts, bs, sd, sf = [], [], [], [], []
    
    return tp,bp,pf

In [ ]:
# calculating thickness of perforated sand in each sand (modified "p" method)
import xlwings as xl

# initialising marker and production data
marker = perfo_wells
prod = xl.Book(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx").sheets['Prod & Comp (Hp)']

j = 0 # starting index for completion data
i = 2 # starting index for production data

# start the machine!
while prod.range(i, 10).value == marker['UWI'][j]:
     
    # if the well in marker and production data is the same 
    if prod.range(i, 10).value == marker['UWI'][j]:

        # create temporary dataframe from marker data that contain unique well
        temp = marker[marker['UWI'] == marker['UWI'][j]]
        
        # initialising x variable and blank array that contain perforation and squeeze data 
        x = 0
        PF,SF,TP,BP,TS,BS,PD,SD = [],[],[],[],[],[],[],[]

        # inserting data to the array
        while x < len(temp):

            # dates in production data is compared to all of the dates data that meet the condition
            if (
                (prod.range(i, 4).value.year > marker['DATE'][x+j].year) or 
                ((prod.range(i, 4).value.year == marker['DATE'][x+j].year) and 
                 (prod.range(i, 4).value.month >= marker['DATE'][x+j].month))
                ):

                # perforation event 
                if marker['STATUS'][x+j] == "perforation":
                    
                    # appending perforation event data
                    PF.append(marker['MARKER'][x+j])
                    TP.append(marker['TOP'][x+j])
                    BP.append(marker['DEPTH'][x+j])
                    PD.append(marker['DATE'][x+j])

                # squeeze event
                elif marker['STATUS'][x+j] == "squeeze":

                    # appending squeeze event data
                    SF.append(marker['MARKER'][x+j])
                    TS.append(marker['TOP'][x+j])
                    BS.append(marker['DEPTH'][x+j])
                    SD.append(marker['DATE'][x+j])
                        
                x += 1

            else:
                break  

        # squeeze handling
        squeeze_machine(TP,BP,PF,PD,TS,BS,SF,SD)

        if len(PF) > 0:

            # removing duplicates
            m = []
            n = list(zip(TP,BP,PF)) 
            [m.append(j) for j in n if j not in m]
            TP,BP,PF = list(zip(*m))

            # calculating thickness of perforation
            sands = {}
            for minas in sand:
                sands[minas] = 0
            
            a = 0
            while a < len(PF):
        
                if len(PF[a]) == 1:
                    sands[PF[a][0]] += BP[a] - TP[a]
                    
                elif len(PF[a]) == 2:
                    sands[PF[a][0]] += marker_wells.loc[marker['UWI'][j],PF[a][1]] - TP[a]
                    sands[PF[a][1]] += BP[a] - marker_wells.loc[marker['UWI'][j],PF[a][1]]
                    
                elif len(PF[a]) == 3:
                    sands[PF[a][0]] += marker_wells.loc[marker['UWI'][j],PF[a][1]] - TP[a]
                    sands[PF[a][1]] += marker_wells.loc[marker['UWI'][j],PF[a][2]] - marker_wells.loc[marker['UWI'][j],PF[a][1]]
                    sands[PF[a][2]] += BP[a] - marker_wells.loc[marker['UWI'][j],PF[a][2]]

                elif len(PF[a]) == 4:
                    sands[PF[a][0]] += marker_wells.loc[marker['UWI'][j],PF[a][1]] - TP[a]
                    sands[PF[a][1]] += marker_wells.loc[marker['UWI'][j],PF[a][2]] - marker_wells.loc[marker['UWI'][j],PF[a][1]]
                    sands[PF[a][2]] += marker_wells.loc[marker['UWI'][j],PF[a][3]] - marker_wells.loc[marker['UWI'][j],PF[a][2]]
                    sands[PF[a][3]] += BP[a] - marker_wells.loc[marker['UWI'][j],PF[a][3]]

                elif len(PF[a]) == 5:
                    sands[PF[a][0]] += marker_wells.loc[marker['UWI'][j],PF[a][1]] - TP[a]
                    sands[PF[a][1]] += marker_wells.loc[marker['UWI'][j],PF[a][2]] - marker_wells.loc[marker['UWI'][j],PF[a][1]]
                    sands[PF[a][2]] += marker_wells.loc[marker['UWI'][j],PF[a][3]] - marker_wells.loc[marker['UWI'][j],PF[a][2]]
                    sands[PF[a][3]] += marker_wells.loc[marker['UWI'][j],PF[a][4]] - marker_wells.loc[marker['UWI'][j],PF[a][3]]             
                    sands[PF[a][4]] += BP[a] - marker_wells.loc[marker['UWI'][j],PF[a][4]]
                
                a += 1

        # write thickness perforation on the excel to mark perforated sand
        writing_perforation(sands, i)
        i += 1
    
    # if the the well is different, move to another well 
    if prod.range(i, 10).value != marker['UWI'][j]:
            
        j += len(temp)

In [ ]:
# splitting machine
import pandas as pd
import xlwings as xw

# create workabook
wb =  xw.Book(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx")
ws = wb.sheets['Split']

# lumping data
lumping_lqr = pd.read_excel(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx", sheet_name = 'Lumping LQR')
lumping_combined = pd.read_excel(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx", sheet_name = 'Lumping LQR+HQR')
# perforation data
perfo = pd.read_excel(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx", sheet_name = 'Prod & Comp (Hp)')
# well data
well = pd.read_excel(r"C:\Users\GG\Downloads\Splitting All Wells (KH).xlsx", sheet_name = 'Well')

row_count = 3
i = 0

for index in well.index:
    
    while i < len(perfo):
        
        if str(perfo['UWI'][i]) == str(well['Well'][index]):
            
            # define total_kh initial
            total_kh = 0
            # define formation count intitial
            perfo_count = 15

            for khsov in lumping_combined.index:
                if perfo.iloc[row_count-3, perfo_count] > 0:
                    total_kh += lumping_combined[str(well['Well'][index])][khsov]
                perfo_count += 1

            perfo_count = 15
            num = 16

            for form in lumping_lqr.index:
                if total_kh > 0:
                    if perfo.iloc[row_count-3, perfo_count] > 0:
                        ws.range(row_count, num).value = float(perfo['CUM OIL'][i])*float(lumping_lqr[str(well['Well'][index])][form])/total_kh
                        ws.range(row_count, num+1).value = float(perfo['CUM WATER'][i])*float(lumping_lqr[str(well['Well'][index])][form])/total_kh
                    else:
                        ws.range(row_count, num).value = 0
                        ws.range(row_count, num+1).value = 0

                    
                perfo_count += 1
                num += 2  

            row_count += 1
            i += 1

        else:
            break